In [44]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import gym
import gym_combrf
import matplotlib.pyplot as plt
from Source.antenna import upa
from Source.misc_fun.utils import *
import mpl_toolkits.mplot3d.axes3d as axes3d

import numpy as np

In [49]:
env = gym.make('combrf-v12')

In [55]:
beams = Generate_UPABeams(4,1,np.array([np.pi/4]))
print(beams)
print(len(beams))
#print(beams[2])
#print(beams[58:63])

[(0.78539816, 4, 0.78539816, 1) (0.78539816, 4, 1.57079633, 1)
 (0.78539816, 4, 2.35619449, 1) (0.78539816, 4, 3.14159265, 1)
 (1.57079633, 4, 0.78539816, 1) (1.57079633, 4, 1.57079633, 1)
 (1.57079633, 4, 2.35619449, 1) (1.57079633, 4, 3.14159265, 1)
 (2.35619449, 4, 0.78539816, 1) (2.35619449, 4, 1.57079633, 1)
 (2.35619449, 4, 2.35619449, 1) (2.35619449, 4, 3.14159265, 1)
 (3.14159265, 4, 0.78539816, 1) (3.14159265, 4, 1.57079633, 1)
 (3.14159265, 4, 2.35619449, 1) (3.14159265, 4, 3.14159265, 1)]
16


In [4]:
#Create a UPA steering vector and plot the 3D beam

beamset = Generate_BeamDir(4)

bs_steer = upa.steervec(4,4, beamset[1],0)
print(bs_steer)

bs_steer2 = upa.steervec(4,4, beamset[1],beamset[1])
print(bs_steer2)

X,Y,Z = upa_plotbeam(beamset[1], 0, 4,4)

#fig = plt.figure()
ax = plt.subplot(111, projection='3d')#fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_surface(
    X, Y, Z, rstride=1, cstride=1, cmap=plt.get_cmap('jet'),
    linewidth=0, antialiased=False, alpha=0.5)

plt.show()
#ax = plt.subplot(111, projection='polar')
##print(theta.shape, gr.shape)
#ax.plot(theta2, gr2)
#plt.show()
    

[ 0.25+0.00000000e+00j -0.25+3.06161700e-17j  0.25-6.12323400e-17j
 -0.25+9.18485099e-17j  0.25+0.00000000e+00j -0.25+3.06161700e-17j
  0.25-6.12323400e-17j -0.25+9.18485099e-17j  0.25+0.00000000e+00j
 -0.25+3.06161700e-17j  0.25-6.12323400e-17j -0.25+9.18485099e-17j
  0.25+0.00000000e+00j -0.25+3.06161700e-17j  0.25-6.12323400e-17j
 -0.25+9.18485099e-17j]
[ 0.25+0.00000000e+00j  0.25+4.80917673e-17j  0.25+9.61835347e-17j
  0.25+1.44275302e-16j -0.25+3.06161700e-17j -0.25-1.74755974e-17j
 -0.25-6.55673647e-17j -0.25-1.13659132e-16j  0.25-6.12323400e-17j
  0.25-1.31405726e-17j  0.25+3.49511947e-17j  0.25+8.30429621e-17j
 -0.25+9.18485099e-17j -0.25+4.37567426e-17j -0.25-4.33502475e-18j
 -0.25-5.24267921e-17j]


In [32]:
ue_loc = np.array([[100,0,25]])
bs_loc = np.array([[0,0,25]])

#channel parameters
from Source.misc_fun.upa_channel_mmW import *
ch_model = 'uma-los'
freq = 30e9
tx_loc = ue_loc
rx_loc = bs_loc
sc_xyz = np.array([])
N_rx = 8
N_tx = 1
nFFT = 1
P_tx = 0 #in dB
df = 60 * 1e3  # 75e3  # carrier spacing frequency
B = nFFT * df #bandwidth

ch_randval = np.exp(1j * 2 * np.pi * 0.3)
channel = Channel(freq, tx_loc, rx_loc, sc_xyz, 'model', ch_model, 'nrx', N_rx,'ntx', N_tx, 'nFFT', nFFT, 'df', df)
channel.generate_paths(ch_randval, 0)
h = channel.get_h()

#noise
N0dBm = -174  # mW/Hz
N0 = db2lin(N0dBm) * (10 ** -3)  # in WHz-1

#Codebook
beamwidth_vec = np.array([np.pi / N_rx])
BeamSet = Generate_Beams(N_rx, beamwidth_vec)

#tx_beam
tx_dir_ndx = 0
tx_bdir = BeamSet[tx_dir_ndx] 
tx_beam = upa.var_steervec(N_tx, N_tx, tx_bdir, tx_bdir)

max_rate = 0.0
for rbdir_ndx1 in range(N_rx):
    rx_bdir1 = BeamSet[rbdir_ndx1]
    for rbdir_ndx2 in range(N_rx):
        rx_bdir2 = BeamSet[rbdir_ndx2]
    
        #print("reset: ", rbdir_ndx)
        wRF = upa.var_steervec(N_rx, N_rx, rx_bdir1 , rx_bdir2)

        eff_ch = np.array(h[:, :, 0]).dot(tx_beam)
        #print(wRF.shape)
        #print(eff_ch.shape)
        #print(tx_beam.shape)
        #print(h.shape)
        rssi_val = np.sqrt(N_rx*N_rx*N_tx*N_tx)*np.array(np.conj(wRF.T).dot(eff_ch)) #+ (np.conj(wRF.T).dot(self.noise))[0]
        Es = db2lin(P_tx)  # * (1e-3 / self.B)
        SNR = Es * np.abs(rssi_val) ** 2 / (N0 * B)
        rate = np.log2(1 + SNR)
        
        print("rate: {}, xaction: {}, yaction: {}".format(rate, rbdir_ndx1, rbdir_ndx2))
        if rate >= max_rate:
            max_rate = rate
            max_xaction_ndx = rbdir_ndx1
            max_yaction_ndx = rbdir_ndx2
            max_rssi_val = rssi_val

print("best_rate: {}, best_xaction: {}, best_yaction: {}".format(max_rate, max_xaction_ndx, max_yaction_ndx))


rate: 21.87511764982132, xaction: 0, yaction: 0
rate: 11.205900733977389, xaction: 0, yaction: 1
rate: 14.446495104998945, xaction: 0, yaction: 2
rate: 0.0, xaction: 0, yaction: 3
rate: 14.44649510499894, xaction: 0, yaction: 4
rate: 11.205900733977401, xaction: 0, yaction: 5
rate: 21.87511764982132, xaction: 0, yaction: 6
rate: 24.162994857830327, xaction: 0, yaction: 7
rate: 16.339587114649852, xaction: 1, yaction: 0
rate: 0.0, xaction: 1, yaction: 1
rate: 12.97053686443371, xaction: 1, yaction: 2
rate: 0.0, xaction: 1, yaction: 3
rate: 12.970536864433704, xaction: 1, yaction: 4
rate: 0.0, xaction: 1, yaction: 5
rate: 16.339587114649852, xaction: 1, yaction: 6
rate: 24.162994857830327, xaction: 1, yaction: 7
rate: 19.45639632393806, xaction: 2, yaction: 0
rate: 12.846237010026549, xaction: 2, yaction: 1
rate: 12.676793149064185, xaction: 2, yaction: 2
rate: 0.0, xaction: 2, yaction: 3
rate: 12.676793149064181, xaction: 2, yaction: 4
rate: 12.84623701002656, xaction: 2, yaction: 5
rat

In [33]:
#plot the best RX beam


#fig = plt.figure()
ax = plt.subplot(111, projection='3d')#fig.add_subplot(1, 1, 1, projection='3d')

#for action_ndx1 in range(N_rx):
#    for action_ndx2 in range(N_rx):
X,Y,Z = upa_varplotbeam(BeamSet[max_xaction_ndx], BeamSet[max_yaction_ndx], N_rx,N_rx)
ax.plot_surface(
    X, Y, Z, rstride=1, cstride=1, cmap=plt.get_cmap('jet'),
    linewidth=0, antialiased=False, alpha=0.5)

#theta =
ax.scatter(1*np.sin(45*np.pi/180)*np.cos(45*np.pi/180), 1*np.sin(45*np.pi/180)*np.sin(45*np.pi/180),1*np.cos(45*np.pi/180), c='blue')

# Tweak the limits and add latex math labels.
ax.set_zlim(-1, 1)
ax.set_xlabel(r'r$\cos(\theta)_\sin(\phi)$')
ax.set_ylabel(r'r$\sin(\theta)_\sin(\phi)$')
ax.set_zlabel(r'$r\cos(\theta)$')
ax.view_init(elev=31,azim=-117)

plt.show()

In [7]:
print(BeamSet)
print([(x[0]*180/np.pi,x[1]) for x in BeamSet])

[(0.78539816, 4) (1.57079633, 4) (2.35619449, 4) (3.14159265, 4)]
[(45.0, 4), (90.0, 4), (135.0, 4), (180.0, 4)]
